## Permutation Test using One-Way ANOVA in CUDA

In [1]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the PATH
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


### Random Dataset Generation

In [2]:
!python -m pip install scikit-learn
!python -m pip install pandas
!python -m pip install scipy

In [3]:
from sklearn.datasets import make_classification

k = 3
N = 10
X, y = make_classification(
    n_samples = N,             # row number
    n_features = 5,            # feature numbers
    n_informative = 3,         # The number of informative features
    n_redundant = 0,           # The number of redundant features
    n_repeated = 0,            # The number of duplicated features
    n_classes = k,             # The number of classes 
    n_clusters_per_class = 1,  # The number of clusters per class
    random_state = 42,         # random seed 
    scale=100                  # scale of the data
)

In [4]:
import pandas as pd

df = pd.concat([pd.DataFrame(X)[[0]], pd.DataFrame(y).astype(int)], axis=1)
df.columns = [0, 1]
df = df.sort_values(1).reset_index().iloc[:,1:]
df.to_csv("dataset.csv", header=False, index=False)

### Exact Permutation Test

#### C Version (Serial)

In [5]:
%%writefile c_exact_perm.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define MAX_LINE 1024

/* EXACT BINOMIAL & MULTINOMIAL FUNCTIONS */
// Compute C(n,k) exactly using 128-bit integers
unsigned long long int binom(unsigned int n, unsigned int k) {
    if (k > n) return 0;
    if (k > n - k) k = n - k;

    unsigned long long int result = 1;
    for (unsigned int i = 1; i <= k; i++) {
        result = result * (n - k + i) / i;
    }
    return result;
}

/* EXACT multinomial coefficient using sequential binomial method */
unsigned long long int getCountPerm(int total_elements, size_t *repeats, int k) {
    unsigned long long int result = 1;
    int remaining = total_elements;

    for (int i = 0; i < k; i++) {
        int ni = repeats[i];
        unsigned long long int c = binom(remaining, ni);
        result *= c;
        remaining -= ni;
    }
    return result;
}

// helper function for swapping values
void Exchange(size_t* data, size_t a, size_t b) {
    size_t temp = data[a];
    data[a] = data[b];
    data[b] = temp;
}

/* PERMUTATION GENERATOR */
int permute(size_t a[], int n) {
    int l, j;
    for (j = --n; j > 0 && a[j-1] >= a[j]; --j) { ; }
    if (j == 0) return 0;
    for (l = n; a[j-1] >= a[l]; --l) { ; }
    Exchange(a, j-1, l);
    while (j < n) { Exchange(a, j++, n--); }
    return 1;
}

/* ONE WAY ANALYSIS OF VARIANCE */
double OneWayAnova(size_t N, size_t k, size_t *n_i, size_t *group, double *feature){
    double *group_ave = calloc(k, sizeof(double));

    double average = 0.0;
    for (int i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    for (int i = 0; i < k; i++) {
        group_ave[i] /= n_i[i];
    }

    /* SUM OF SQUARED ERROR (SSE) */
    double SSE = 0.0;
    double temp;
    for (int i = 0; i < N; i++) {
        temp = feature[i] - group_ave[group[i]];
        SSE += temp*temp;
    }

    /* SSR (SUM OF SQUARED RESIDUALS) */
    double SSR = 0.0;
    for (int i = 0; i < k; i++) {
        temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }

    free(group_ave);

    /* F-statistic */
    return (SSR/(k-1))/(SSE/(N-k));
}

int main() {
    size_t N;
    size_t k;
    clock_t start, end;
    size_t counter = 10;

    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);

    double *feature = malloc(N * sizeof(double));
    size_t *group = malloc(N * sizeof(size_t));
    size_t *group_copy = malloc(N * sizeof(size_t));
    size_t *perm_array = malloc(N * 10 * sizeof(size_t));
    size_t *n_i = calloc(k, sizeof(size_t));

    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL) {
        perror("Error opening file");
        return 1;
    }

    char line[MAX_LINE];
    size_t i = 0;

    while (fgets(line, sizeof(line), fp)) {
        if (i >= N) break;

        line[strcspn(line, "\n")] = 0;
        char *token = strtok(line, ",");
        int j = 0;

        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token);
            else {
                group[i] = atoi(token);
                if (group[i] >= k){
                    perror("Error group count");
                    return 1;
                }
                n_i[group[i]] += 1;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);

    memcpy(group_copy, group, N * sizeof(size_t));

    /* EXACT PERMUTATION COUNT*/
    unsigned long long int perm_count = getCountPerm(N, n_i, k);

    // Can overflow for large n
    double *F_dist = malloc(perm_count * sizeof(double));

    // Execution time start here: CPU Permutation
    /* CPU PERMUTATION */
    double elapse = 0.0f, 
        time_taken;

    /* PERMUTATION TEST */
    for (int c=0; c<counter; c++){
        start = clock();
        size_t p = 0;
        for (i = 0; i < perm_count; i++){
            // compute One Way ANOVA
            F_dist[i] = OneWayAnova(N, k, n_i, group, feature);

            // save the permutation for sanity check
            if (i < 5 || i > perm_count - 6) {
            memcpy(&perm_array[p * N], group, N * sizeof(size_t));
            p++;
            }

            // change grouping assignment
            permute(group, N);
        }
        end = clock();
        time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
        memcpy(group, group_copy, N * sizeof(size_t));
    }

    FILE *fptr;
    fptr = fopen("c_exact_perm.csv", "w");

    printf("\nFunction (in C) average time for %lu loops is %f milliseconds to execute an array size of %llu permutations.\n", counter, elapse/counter, perm_count);
    printf("\n");

    // Print first 5 and last 5 permutations
    printf("Printing First 5 and Last 5 Permutations\n");
    for (size_t i = 0; i < 5; i++) {
        printf("CPU Permutation %zu: ", i+1);
        for (size_t j = 0; j < N; j++) {
            printf("%zu ", perm_array[i*N + j]);
            fprintf(fptr, "%zu,", perm_array[i*N + j]);
        }
        printf("\n");
        fprintf(fptr, "\n");
    }
    printf("=================================\n");
    for (size_t i = 5; i < 10; i++) {
        printf("CPU Permutation %zu: ", i+1);
        for (size_t j = 0; j < N; j++) {
            printf("%zu ", perm_array[i*N + j]);
            fprintf(fptr, "%zu,", perm_array[i*N + j]);
        }
        printf("\n");
        fprintf(fptr, "\n");
    }

    printf("\nPrinting First 5 and Last 5 Results\n");
    for (int i = 0; i < 5; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf ("F_dist %d: %lf\n", i+1, F_dist[i]);
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf ("F_dist %d: %lf\n", i+1, F_dist[i]);
    } 

    size_t extreme_count = 0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
    }
    }

    // Calculating the p-value for the permutation test
    double p_value = (double)extreme_count/perm_count;
    printf("\nNull F: %lf\n", F_dist[0]);
    printf ("Extreme Count: %lu\n", extreme_count);
    p_value = (double)extreme_count / perm_count;
    printf("p-value: %lf\n", p_value);

    // saving extreme count and p-value
    fprintf(fptr, "%zu,%lf\n", extreme_count, p_value);
    fclose(fptr);

    // free the allocated memory
    free(feature);
    free(group);
    free(n_i);
    free(F_dist);

    return 0;
}

Overwriting c_exact_perm.c


In [6]:
%%bash
gcc c_exact_perm.c -o c_exact_perm -lm

In [7]:
%%bash
./c_exact_perm < input.txt

Number of Rows: Number of Groups: 
Function (in C) average time for 10 loops is 12.726600 milliseconds to execute an array size of 34650 permutations.

Printing First 5 and Last 5 Permutations
CPU Permutation 1: 0 0 0 0 1 1 1 1 2 2 2 2 
CPU Permutation 2: 0 0 0 0 1 1 1 2 1 2 2 2 
CPU Permutation 3: 0 0 0 0 1 1 1 2 2 1 2 2 
CPU Permutation 4: 0 0 0 0 1 1 1 2 2 2 1 2 
CPU Permutation 5: 0 0 0 0 1 1 1 2 2 2 2 1 
CPU Permutation 6: 2 2 2 2 1 1 1 0 0 0 0 1 
CPU Permutation 7: 2 2 2 2 1 1 1 0 0 0 1 0 
CPU Permutation 8: 2 2 2 2 1 1 1 0 0 1 0 0 
CPU Permutation 9: 2 2 2 2 1 1 1 0 1 0 0 0 
CPU Permutation 10: 2 2 2 2 1 1 1 1 0 0 0 0 

Printing First 5 and Last 5 Results
F_dist 1: 2.471594
F_dist 2: 4.272245
F_dist 3: 2.922043
F_dist 4: 2.430135
F_dist 5: 4.092654
F_dist 34646: 4.092654
F_dist 34647: 2.430135
F_dist 34648: 2.922043
F_dist 34649: 4.272245
F_dist 34650: 2.471594

Null F: 2.471594
Extreme Count: 5165
p-value: 0.149062


#### CUDA (using lexicographic permutation)

In [8]:
%%writefile cuda_exact_lexico_perm_anova.cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define MAX_LINE 1024

/* EXACT BINOMIAL & MULTINOMIAL FUNCTIONS */
// Compute C(n,k) exactly using 128-bit integers
unsigned long long binom(unsigned long long n, unsigned long long k) {
    if (k > n) return 0;
    if (k > n - k) k = n - k;

    unsigned long long result = 1;
    for (int i = 1; i <= k; i++) {
        result = result * (n - k + i) / i;
    }
    return result;
}

/* EXACT multinomial coefficient using sequential binomial method */
unsigned long long getCountPerm(int total_elements, size_t *repeats, int k) {
    unsigned long long result = 1;
    int remaining = total_elements;

    for (int i = 0; i < k; i++) {
        int ni = repeats[i];
        unsigned long long c = binom(remaining, ni);
        result *= c;
        remaining -= ni;
    }
    return result;
}

// helper function for swapping values
void Exchange(size_t* data, size_t a, size_t b) {
    size_t temp = data[a];
    data[a] = data[b];
    data[b] = temp;
}

/* PERMUTATION GENERATOR */
int permute(size_t a[], size_t n) {
    int l, j;
    for (j = --n; j > 0 && a[j-1] >= a[j]; --j) { ; }
    if (j == 0) return 0;
    for (l = n; a[j-1] >= a[l]; --l) { ; }
    Exchange(a, j-1, l);
    while (j < n) { Exchange(a, j++, n--); }
    return 1;
}

/* One Way ANOVA */
__device__ double OneWayAnova(size_t N, size_t k, size_t *n_i, size_t *group, double *feature){
    double group_ave[100];
    for (int i = 0; i < k; i++) {
        group_ave[i] = 0.0;
    }
        
    double average = 0.0;
    for (int i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    for (int i = 0; i < k; i++) {
        group_ave[i] /= n_i[i];
    }

    /* SUM OF SQUARED ERROR (SSE) */
    double SSE = 0.0;
    double temp;
    for (int i = 0; i < N; i++) {
        temp = feature[i] - group_ave[group[i]];
        SSE += temp*temp;
    }

    /* SSR (SUM OF SQUARED RESIDUALS) */
    double SSR = 0.0;
    for (int i = 0; i < k; i++) {
        temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }

    /* F-statistic */
    return (SSR/(k-1))/(SSE/(N-k));
}

__global__ void gpu_anova(size_t *perm_array, size_t N, int k, size_t perm_count, double *feature, size_t *n_i, double *F_dist) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int perm_idx = idx; perm_idx < perm_count; perm_idx += stride) {
        size_t *current_group = &perm_array[perm_idx * N];
        F_dist[perm_idx] = OneWayAnova(N, k, n_i, current_group, feature);
    }
}

int main() {
    size_t N;
    size_t k;
    clock_t start, end;
    size_t counter = 10;

    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);

    // Get GPU device
    int device = -1;
    cudaGetDevice(&device);

    // Memory allocation
    double *feature;
    size_t *group;
    size_t *group_copy = (size_t*)malloc(N * sizeof(size_t));
    size_t *n_i;
    size_t *perm_array;
    double *F_dist;

    cudaMallocManaged(&feature, N * sizeof(double));
    cudaMallocManaged(&group, N * sizeof(size_t));
    cudaMallocManaged(&n_i, k * sizeof(size_t));

    // Initialize n_i to zero
    memset(n_i, 0, k * sizeof(size_t));

    // MEMORY ADVISE: Set up for input data (feature, group, n_i)
    cudaMemAdvise(feature, N * sizeof(double), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(group, N * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(n_i, k * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);

    // Prefetch data to CPU memory
    cudaMemPrefetchAsync(feature, N * sizeof(double), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), cudaCpuDeviceId, NULL);

    // READ DATA FROM FILE
    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL){
        perror("Error opening file");
        return 1;
    }

    char line[MAX_LINE];
    size_t i = 0;
    while (fgets(line, sizeof(line), fp)) {
        if (i >= N) break;

        line[strcspn(line, "\n")] = 0;

        char *token = strtok(line, ",");
        int j = 0;
        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token);
            else {
                group[i] = atoi(token);
                if (group[i] >= k){
                    perror("Error group count");
                    fclose(fp);
                    return 1;
                }
                n_i[group[i]] += 1;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);    

    memcpy(group_copy, group, N * sizeof(size_t));

    /* EXACT PERMUTATION COUNT USING 128-BIT INTEGER */
    unsigned long long perm_count = getCountPerm(N, n_i, k);
    
    cudaMallocManaged(&perm_array, N * perm_count * sizeof(size_t));
    cudaMallocManaged(&F_dist, perm_count * sizeof(double));

    /* CPU PERMUTATION */
    double elapse = 0.0f, 
           time_taken;

    // STEP 1: CPU PERMUTATION
    for (int c = 0; c < counter; c++){
        start = clock();
        memcpy(perm_array, group, N * sizeof(size_t)); // Initialize first permutation
        for (i = 0; i < perm_count; i++) {
            permute(group, N);
            memcpy(&perm_array[(i + 1) * N], group, N * sizeof(size_t));
        }
        memcpy(group, group_copy, N * sizeof(size_t)); // Reset group array
        end = clock();
        time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
    }
    printf("\nFunction (in C) average time for %lu loops is %f milliseconds to generate %llu permutations\n", counter, elapse/counter, perm_count);

    // PREFETCH: Move input data to GPU before computation
    cudaMemPrefetchAsync(feature, N * sizeof(double), device, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), device, NULL);
    
    // Prefetch output arrays to GPU
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), device, NULL);

    // Wait for prefetch to complete
    cudaDeviceSynchronize();

    // Number of Threads and Blocks
    size_t numThreads = 256;
    size_t numBlocks = (perm_count + numThreads - 1) / numThreads;
    
    for (size_t c = 0; c < counter; c++){
        gpu_anova<<<numBlocks, numThreads>>>(perm_array, N, k, perm_count, feature, n_i, F_dist);
    }
    cudaDeviceSynchronize();

    // PREFETCH: Move results back to CPU for printing
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), cudaCpuDeviceId, NULL);

    // PRINT RESULTS
    FILE *fptr;
    fptr = fopen("cuda_exact_lexico_perm_anova.csv", "w");

    printf("\nPrinting First 5 and Last 5 permutations\n");
    for (int i = 0; i < 5; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }

    printf("\nPrinting First 5 and Last 5 F-statistics:\n");
    for (int i = 0; i < 5; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist[%d]: %lf\n", i, F_dist[i]);
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }

    // Calculate p-value
    size_t extreme_count = 0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
        }
    }
    double p_value = (double)extreme_count / (double)perm_count;
    printf("\nNull: %lf\n", F_dist[0]);
    printf("Extreme count: %zu\n", extreme_count);
    printf("p-value: %lf\n", p_value);

    fprintf(fptr, "%zu,%lf", extreme_count, p_value);

    // Free memory
    cudaFree(feature);
    cudaFree(group);
    cudaFree(n_i);
    cudaFree(perm_array);
    cudaFree(F_dist);

    return 0;
}

Overwriting cuda_exact_lexico_perm_anova.cu


In [9]:
%%bash
nvcc cuda_exact_lexico_perm_anova.cu -o cuda_exact_lexico_perm_anova -Wno-deprecated-gpu-targets

In [10]:
%%bash
nvprof ./cuda_exact_lexico_perm_anova < input.txt

==403799== NVPROF is profiling process 403799, command: ./cuda_exact_lexico_perm_anova


Number of Rows: Number of Groups: 
Function (in C) average time for 10 loops is 3.781600 milliseconds to generate 34650 permutations

Printing First 5 and Last 5 permutations
GPU Permutation 1: 0 0 0 0 1 1 1 1 2 2 2 2 
GPU Permutation 2: 0 0 0 0 1 1 1 2 1 2 2 2 
GPU Permutation 3: 0 0 0 0 1 1 1 2 2 1 2 2 
GPU Permutation 4: 0 0 0 0 1 1 1 2 2 2 1 2 
GPU Permutation 5: 0 0 0 0 1 1 1 2 2 2 2 1 
GPU Permutation 34646: 2 2 2 2 1 1 1 0 0 0 0 1 
GPU Permutation 34647: 2 2 2 2 1 1 1 0 0 0 1 0 
GPU Permutation 34648: 2 2 2 2 1 1 1 0 0 1 0 0 
GPU Permutation 34649: 2 2 2 2 1 1 1 0 1 0 0 0 
GPU Permutation 34650: 2 2 2 2 1 1 1 1 0 0 0 0 

Printing First 5 and Last 5 F-statistics:
F_dist[0]: 2.471594
F_dist[1]: 4.272245
F_dist[2]: 2.922043
F_dist[3]: 2.430135
F_dist[4]: 4.092654
F_dist 34646: 4.092654
F_dist 34647: 2.430135
F_dist 34648: 2.922043
F_dist 34649: 4.272245
F_dist 34650: 2.471594

Null: 2.471594
Extreme count: 5165
p-value: 0.149062


==403799== Profiling application: ./cuda_exact_lexico_perm_anova
==403799== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  140.64us        10  14.064us  13.408us  18.144us  gpu_anova(unsigned long*, unsigned long, int, unsigned long, double*, unsigned long*, double*)
      API calls:   98.69%  818.38ms         5  163.68ms  29.358us  817.90ms  cudaMallocManaged
                    0.57%  4.7124ms        10  471.24us  18.307us  1.2934ms  cudaMemPrefetchAsync
                    0.22%  1.8640ms        10  186.40us  6.9480us  1.7735ms  cudaLaunchKernel
                    0.18%  1.4984ms         5  299.68us  31.830us  760.72us  cudaFree
                    0.10%  848.56us         2  424.28us  54.005us  794.55us  cudaDeviceSynchronize
                    0.08%  660.46us       114  5.7930us     135ns  254.97us  cuDeviceGetAttribute
                    0.08%  659.85us         1  659.85us  659.85us  659.85us  cudaG

#### CUDA Version (using rank indexing)

In [11]:
%%writefile cuda_exact_rank_perm_anova.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define MAX_LINE 1024
#define MAX_GROUPS 100

__device__ unsigned long long factorial(size_t n) {
    unsigned long long result = 1;
    for (int i = 2; i <= n; i++)
        result *= i;
    return result;
}

__device__ unsigned long long multinomial(size_t total, size_t *counts, size_t k) {
    unsigned long long result = factorial(total);
    for (int i = 0; i < k; i++) {
        result /= factorial(counts[i]);
    }
    return result;
}

__device__ void rank_to_permutation(size_t *keys, size_t *n_i, size_t k, size_t N, unsigned long long rank, size_t *perm) {
    size_t n_i_copy[MAX_GROUPS];
    
    for (int i = 0; i < k; i++) {
        n_i_copy[i] = n_i[i];
    }
    
    int total = N;
    
    for (int pos = 0; pos < N; pos++) {
        for (int i = 0; i < k; i++) {
            if (n_i_copy[i] == 0)
                continue;
            
            n_i_copy[i]--;
            unsigned long long num = multinomial(total - 1, n_i_copy, k);
            
            if (rank < num) {
                perm[pos] = keys[i];
                total--;
                break;
            } else {
                rank -= num;
                n_i_copy[i]++;
            }
        }
    }
}

__device__ double one_way_anova(size_t N, size_t k, size_t *n_i, size_t *group, double *feature) {
    double group_ave[MAX_GROUPS] = {0.0};
    
    double average = 0.0;
    for (int i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    
    for (int i = 0; i < k; i++) {
        if (n_i[i] > 0)
            group_ave[i] /= n_i[i];
    }
    
    double SSE = 0.0;
    for (int i = 0; i < N; i++) {
        double temp = feature[i] - group_ave[group[i]];
        SSE += temp * temp;
    }
    
    double SSR = 0.0;
    for (int i = 0; i < k; i++) {
        double temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }
    
    return (SSR / (k - 1)) / (SSE / (N - k));
}

__global__ void permutation_test_gpu(size_t N, size_t k, size_t *keys, size_t *n_i, double *features, unsigned long long perm_count, size_t *perm_array,  double *F_dist) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (unsigned long long perm_idx = idx; perm_idx < perm_count; perm_idx += stride) {
        unsigned long long rank = perm_idx;
        
        // Each thread gets its own section of the perm_array
        size_t *perm = &perm_array[idx * N];
        if (rank >= perm_count)
            return;
        
        rank_to_permutation(keys, n_i, k, N, rank, perm);
        
        double F_stat = one_way_anova(N, k, n_i, perm, features);
        
        F_dist[rank] = F_stat;
    }
}

unsigned long long binom(size_t n, size_t k) {
    if (k > n) return 0;
    if (k > n - k) k = n - k;
    
    unsigned long long result = 1;
    for (int i = 1; i <= k; i++) {
        result = result * (n - k + i) / i;
    }
    return result;
}

unsigned long long get_perm_count(size_t total_elements, size_t *n_i, size_t k) {
    unsigned long long result = 1;
    int remaining = total_elements;
    
    for (int i = 0; i < k; i++) {
        int ni = n_i[i];
        unsigned long long c = binom(remaining, ni);
        result *= c;
        remaining -= ni;
    }
    return result;
}

int main() {
    size_t N, k;
    size_t counter = 10;
    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);

    int device = -1;
    cudaGetDevice(&device);

    double *feature;
    size_t *group;
    size_t *n_i;
    size_t *keys;
    double *F_dist;
    size_t *perm_array;

    cudaMallocManaged(&feature, N * sizeof(double));
    cudaMallocManaged(&group, N * sizeof(size_t));
    cudaMallocManaged(&n_i, k * sizeof(size_t));
    cudaMallocManaged(&keys, k * sizeof(size_t));
    
    memset(n_i, 0, k * sizeof(size_t));

    cudaMemAdvise(feature, N * sizeof(double), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(group, N * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(n_i, k * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(keys, k * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);

    cudaMemPrefetchAsync(feature, N * sizeof(double), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(keys, k * sizeof(size_t), cudaCpuDeviceId, NULL);
    
    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL) {
        perror("Error opening file");
        return 1;
    }
    
    char line[MAX_LINE];
    int i = 0;
    
    while (fgets(line, sizeof(line), fp) && i < N) {
        line[strcspn(line, "\n")] = 0;
        char *token = strtok(line, ",");
        int j = 0;
        
        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token);
            else {
                group[i] = atoi(token);
                if (group[i] >= k) {
                    fprintf(stderr, "Error: group index out of range\n");
                    fclose(fp);
                    return 1;
                }
                n_i[group[i]]++;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);
    
    for (size_t i = 0; i < k; i++) {
        keys[i] = i;
    }
    
    unsigned long long perm_count = get_perm_count(N, n_i, k);

    cudaMallocManaged(&F_dist, perm_count * sizeof(double));
    cudaMallocManaged(&perm_array, perm_count * N * sizeof(size_t));

    cudaMemPrefetchAsync(keys, k * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(feature, N * sizeof(double), device, NULL);
    
    // Prefetch output array to GPU
    cudaMemPrefetchAsync(perm_array, perm_count * N * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), device, NULL);
    
    size_t numThreads = 256;
    size_t numBlocks = (perm_count + numThreads - 1) / numThreads;

    for (size_t c = 0; c < counter; c++){
        permutation_test_gpu<<<numBlocks, numThreads>>>(
            N, k, keys, n_i, feature, perm_count, perm_array, F_dist
        );
    }
    
    cudaDeviceSynchronize();
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), cudaCpuDeviceId, NULL);
    
    // PRINT RESULTS
    FILE *fptr;
    fptr = fopen("cuda_exact_rank_perm_anova.csv", "w");

    printf("\nPrinting First 5 and Last 5 permutations\n");
    for (int i = 0; i < 5; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }

    printf("\nPrinting First 5 and Last 5 F-statistics:\n");
    for (int i = 0; i < 5; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }

    // Calculate p-value
    size_t extreme_count = 0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
        }
    }
    double p_value = (double)extreme_count / (double)perm_count;
    printf("\nNull: %lf\n", F_dist[0]);
    printf("Extreme count: %zu\n", extreme_count);
    printf("p-value: %lf\n", p_value);

    fprintf(fptr, "%zu,%lf", extreme_count, p_value);

    cudaFree(feature);
    cudaFree(group);
    cudaFree(n_i);
    cudaFree(keys);
    cudaFree(F_dist);

    return 0;
}

Overwriting cuda_exact_rank_perm_anova.cu


In [12]:
%%bash
nvcc cuda_exact_rank_perm_anova.cu -o cuda_exact_rank_perm_anova -Wno-deprecated-gpu-targets

In [13]:
%%bash
nvprof ./cuda_exact_rank_perm_anova < input.txt

==403863== NVPROF is profiling process 403863, command: ./cuda_exact_rank_perm_anova


Number of Rows: Number of Groups: 
Printing First 5 and Last 5 permutations
GPU Permutation 1: 0 0 0 0 1 1 1 1 2 2 2 2 
GPU Permutation 2: 0 0 0 0 1 1 1 2 1 2 2 2 
GPU Permutation 3: 0 0 0 0 1 1 1 2 2 1 2 2 
GPU Permutation 4: 0 0 0 0 1 1 1 2 2 2 1 2 
GPU Permutation 5: 0 0 0 0 1 1 1 2 2 2 2 1 
GPU Permutation 34646: 2 2 2 2 1 1 1 0 0 0 0 1 
GPU Permutation 34647: 2 2 2 2 1 1 1 0 0 0 1 0 
GPU Permutation 34648: 2 2 2 2 1 1 1 0 0 1 0 0 
GPU Permutation 34649: 2 2 2 2 1 1 1 0 1 0 0 0 
GPU Permutation 34650: 2 2 2 2 1 1 1 1 0 0 0 0 

Printing First 5 and Last 5 F-statistics:
F_dist 1: 2.471594
F_dist 2: 4.272245
F_dist 3: 2.922043
F_dist 4: 2.430135
F_dist 5: 4.092654
F_dist 34646: 4.092654
F_dist 34647: 2.430135
F_dist 34648: 2.922043
F_dist 34649: 4.272245
F_dist 34650: 2.471594

Null: 2.471594
Extreme count: 5165
p-value: 0.149062


==403863== Profiling application: ./cuda_exact_rank_perm_anova
==403863== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  1.1662ms        10  116.62us  114.08us  118.30us  permutation_test_gpu(unsigned long, unsigned long, unsigned long*, unsigned long*, double*, __int64, unsigned long*, double*)
      API calls:   99.40%  1.05655s         6  176.09ms  7.7000us  1.05604s  cudaMallocManaged
                    0.14%  1.4663ms        10  146.63us  16.271us  334.48us  cudaMemPrefetchAsync
                    0.13%  1.3915ms        10  139.15us  4.8540us  1.3201ms  cudaLaunchKernel
                    0.10%  1.0870ms         1  1.0870ms  1.0870ms  1.0870ms  cudaDeviceSynchronize
                    0.09%  1.0071ms         5  201.42us  20.450us  708.16us  cudaFree
                    0.04%  471.55us       114  4.1360us     137ns  206.26us  cuDeviceGetAttribute
                    0.04%  395.79us         1  395.79

#### CUDA Version (using rank indexing w/ shared memory)

In [14]:
%%writefile cuda_shared_exact_rank_perm_anova.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define MAX_LINE 1024
#define MAX_GROUPS 10

__device__ unsigned long long factorial(size_t n) {
    unsigned long long result = 1;
    for (size_t i = 2; i <= n; i++)
        result *= i;
    return result;
}

__device__ unsigned long long multinomial(size_t total, size_t *counts, size_t k) {
    unsigned long long result = factorial(total);
    for (size_t i = 0; i < k; i++) {
        result /= factorial(counts[i]);
    }
    return result;
}

__device__ void rank_to_permutation(size_t *keys, size_t *n_i, size_t k, size_t N, unsigned long long rank, size_t *perm) {
    size_t n_i_copy[MAX_GROUPS];
    
    for (int i = 0; i < k; i++) {
        n_i_copy[i] = n_i[i];
    }
    
    size_t total = N;
    
    for (int pos = 0; pos < N; pos++) {
        for (int i = 0; i < k; i++) {
            if (n_i_copy[i] == 0)
                continue;
            
            n_i_copy[i]--;
            unsigned long long num = multinomial(total - 1, n_i_copy, k);
            
            if (rank < num) {
                perm[pos] = keys[i];
                total--;
                break;
            } else {
                rank -= num;
                n_i_copy[i]++;
            }
        }
    }
}

__device__ double one_way_anova(size_t N, size_t k, size_t *n_i, size_t *group, double *feature) {
    double group_ave[MAX_GROUPS] = {0.0};
    
    double average = 0.0;
    for (size_t i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    
    for (size_t i = 0; i < k; i++) {
        if (n_i[i] > 0)
            group_ave[i] /= n_i[i];
    }
    
    double SSE = 0.0;
    for (size_t i = 0; i < N; i++) {
        double temp = feature[i] - group_ave[group[i]];
        SSE += temp * temp;
    }
    
    double SSR = 0.0;
    for (size_t i = 0; i < k; i++) {
        double temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }
    
    return (SSR / (k - 1)) / (SSE / (N - k));
}

__global__ void permutation_test_gpu(size_t N, size_t k, size_t *keys, size_t *group_counts, double *features, unsigned long long total_perms, size_t *perm_buffer,  double *F_dist) {
    extern __shared__ char shared_mem[];
    double* shared_feature = (double*)shared_mem;
    size_t* shared_keys = (size_t*)(shared_mem + N * sizeof(double));
    size_t* shared_group_counts = (size_t*)(shared_mem + N * sizeof(double) + k * sizeof(size_t));

    int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    int lindex = threadIdx.x;

    // Load data into shared memory
    for (int i = lindex; i < N; i += blockDim.x) {
        shared_feature[i] = features[i];
        if (i < k) {
            shared_keys[i] = keys[i];
            shared_group_counts[i] = group_counts[i];
        }
    }

    __syncthreads();
    int stride = blockDim.x * gridDim.x;
    for (unsigned long long perm_idx = thread_id; perm_idx < total_perms; perm_idx += stride) {
        unsigned long long rank = perm_idx;
        
        // Each thread gets its own section of the perm_buffer
        size_t *perm = &perm_buffer[perm_idx * N];
        if (rank >= total_perms)
            return;
        
        rank_to_permutation(shared_keys, shared_group_counts, k, N, rank, perm);
        
        double F_stat = one_way_anova(N, k, shared_group_counts, perm, shared_feature);
        
        F_dist[rank] = F_stat;
    }
}

unsigned long long binom(size_t n, size_t k) {
    if (k > n) return 0;
    if (k > n - k) k = n - k;
    
    unsigned long long result = 1;
    for (size_t i = 1; i <= k; i++) {
        result = result * (n - k + i) / i;
    }
    return result;
}

unsigned long long get_perm_count(size_t total_elements, size_t *repeats, size_t k) {
    unsigned long long result = 1;
    int remaining = total_elements;
    
    for (int i = 0; i < k; i++) {
        int ni = repeats[i];
        unsigned long long c = binom(remaining, ni);
        result *= c;
        remaining -= ni;
    }
    return result;
}

int main() {
    size_t N, k;
    size_t counter = 10;
    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);

    int device = -1;
    cudaGetDevice(&device);

    double *feature;
    size_t *group;
    size_t *n_i;
    size_t *keys;
    double *F_dist;
    size_t *perm_array;

    cudaMallocManaged(&feature, N * sizeof(double));
    cudaMallocManaged(&group, N * sizeof(size_t));
    cudaMallocManaged(&n_i, k * sizeof(size_t));
    cudaMallocManaged(&keys, k * sizeof(size_t));

    memset(n_i, 0, k * sizeof(size_t));

    cudaMemAdvise(feature, N * sizeof(double), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(group, N * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(n_i, k * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(keys, k * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);

    cudaMemPrefetchAsync(feature, N * sizeof(double), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(keys, k * sizeof(size_t), cudaCpuDeviceId, NULL);
    
    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL) {
        perror("Error opening file");
        return 1;
    }
    
    char line[MAX_LINE];
    int i = 0;
    
    while (fgets(line, sizeof(line), fp) && i < N) {
        line[strcspn(line, "\n")] = 0;
        char *token = strtok(line, ",");
        int j = 0;
        
        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token);
            else {
                group[i] = atoi(token);
                if (group[i] >= k) {
                    fprintf(stderr, "Error: group index out of range\n");
                    fclose(fp);
                    return 1;
                }
                n_i[group[i]]++;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);
    
    for (int i = 0; i < k; i++) {
        keys[i] = i;
    }
    
    unsigned long long perm_count = get_perm_count(N, n_i, k);

    cudaMallocManaged(&F_dist, perm_count * sizeof(double));
    cudaMallocManaged(&perm_array, perm_count * N * sizeof(size_t));

    cudaMemPrefetchAsync(keys, k * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(feature, N * sizeof(double), device, NULL);
    
    // Prefetch output array to GPU
    cudaMemPrefetchAsync(perm_array, perm_count * N * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), device, NULL);
    
    size_t numThreads = 256;
    size_t numBlocks = (perm_count + numThreads - 1) / numThreads;
    
    for (size_t c = 0; c < counter; c++){
        permutation_test_gpu<<<numBlocks, numThreads, N + 2*k>>>(
            N, k, keys, n_i, feature, perm_count, perm_array, F_dist
        );
    }
    
    cudaDeviceSynchronize();
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), cudaCpuDeviceId, NULL);
    
    // PRINT RESULTS
    FILE *fptr;
    fptr = fopen("cuda_shared_exact_rank_perm_anova.csv", "w");

    printf("\nPrinting First 5 and Last 5 permutations\n");
    for (int i = 0; i < 5; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }

    printf("\nPrinting First 5 and Last 5 F-statistics:\n");
    for (int i = 0; i < 5; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }

    // Calculate p-value
    size_t extreme_count = 0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
        }
    }
    double p_value = (double)extreme_count / (double)perm_count;
    printf("\nNull: %lf\n", F_dist[0]);
    printf("Extreme count: %zu\n", extreme_count);
    printf("p-value: %lf\n", p_value);

    fprintf(fptr, "%zu,%lf", extreme_count, p_value);

    cudaFree(feature);
    cudaFree(group);
    cudaFree(n_i);
    cudaFree(keys);
    cudaFree(F_dist);

    return 0;
}

Overwriting cuda_shared_exact_rank_perm_anova.cu


In [15]:
%%bash
nvcc cuda_shared_exact_rank_perm_anova.cu -o cuda_shared_exact_rank_perm_anova -Wno-deprecated-gpu-targets

In [16]:
%%bash
nvprof ./cuda_shared_exact_rank_perm_anova < input.txt

==403935== NVPROF is profiling process 403935, command: ./cuda_shared_exact_rank_perm_anova


Number of Rows: Number of Groups: 
Printing First 5 and Last 5 permutations
GPU Permutation 1: 0 0 0 0 1 1 1 1 2 2 2 2 
GPU Permutation 2: 0 0 0 0 1 1 1 2 1 2 2 2 
GPU Permutation 3: 0 0 0 0 1 1 1 2 2 1 2 2 
GPU Permutation 4: 0 0 0 0 1 1 1 2 2 2 1 2 
GPU Permutation 5: 0 0 0 0 1 1 1 2 2 2 2 1 
GPU Permutation 34646: 2 2 2 2 1 1 1 0 0 0 0 1 
GPU Permutation 34647: 2 2 2 2 1 1 1 0 0 0 1 0 
GPU Permutation 34648: 2 2 2 2 1 1 1 0 0 1 0 0 
GPU Permutation 34649: 2 2 2 2 1 1 1 0 1 0 0 0 
GPU Permutation 34650: 2 2 2 2 1 1 1 1 0 0 0 0 

Printing First 5 and Last 5 F-statistics:
F_dist 1: 2.471594
F_dist 2: 4.272245
F_dist 3: 2.922043
F_dist 4: 2.430135
F_dist 5: 4.092654
F_dist 34646: 4.092654
F_dist 34647: 2.430135
F_dist 34648: 2.922043
F_dist 34649: 4.272245
F_dist 34650: 2.471594

Null: 2.471594
Extreme count: 5165
p-value: 0.149062


==403935== Profiling application: ./cuda_shared_exact_rank_perm_anova
==403935== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  677.18us        10  67.718us  65.727us  70.848us  permutation_test_gpu(unsigned long, unsigned long, unsigned long*, unsigned long*, double*, __int64, unsigned long*, double*)
      API calls:   99.38%  1.30376s         6  217.29ms  10.238us  1.30308s  cudaMallocManaged
                    0.18%  2.3497ms        10  234.97us  8.2930us  2.2144ms  cudaLaunchKernel
                    0.18%  2.3459ms        10  234.59us  18.515us  649.97us  cudaMemPrefetchAsync
                    0.11%  1.3947ms         5  278.94us  32.403us  957.85us  cudaFree
                    0.04%  516.85us         1  516.85us  516.85us  516.85us  cudaDeviceSynchronize
                    0.04%  483.09us       114  4.2370us     133ns  195.90us  cuDeviceGetAttribute
                    0.03%  424.66us         4 

In [17]:
import numpy as np
from scipy.stats import f_oneway

features = pd.read_csv("dataset.csv",header=None)[0].tolist()
filenames = [
    'c_exact_perm.csv', 
    'cuda_exact_lexico_perm_anova.csv', 
    'cuda_exact_rank_perm_anova.csv', 
    'cuda_shared_exact_rank_perm_anova.csv'
]

for filename in filenames:
    file_object = open(filename)
    content = file_object.read()
    permutations = []
    f_stats = []
    indices = []
    p_value = 0.0
    extreme_count = 0
    for i, row in enumerate(content.split('\n')):
        if i < 10:
            permutations.append(row[:-1]) 
        elif i < 20:
            index, f = row.split(',')
            indices.append(int(index))
            f_stats.append(float(f))
        elif row != '':
            extreme_count, p_value = row.split(',')
            extreme_count = int(extreme_count)
            p_value = float(p_value)
    permutations = np.array(permutations)
    f_stats = np.array(f_stats)
    indices = np.array(indices)
    output_df = pd.DataFrame(np.vstack([indices, permutations, f_stats])).T
    output_df.columns = ['i', 'perm', 'f']
    output_df['i'] = output_df['i'].astype(int)
    output_df['f'] = output_df['f'].astype(float)
    actual_fs = []
    for perm in output_df['perm']:
        permuted_df = pd.DataFrame(np.vstack([np.array(perm.split(',')), features])).T
        permuted_df[0] = permuted_df[0].astype(int)
        permuted_df[1] = permuted_df[1].astype(float)
        keys = permuted_df[0].unique().tolist()
        input_features = []
        for key in keys:
            group_feature = permuted_df[permuted_df[0] == key][1].tolist()
            input_features.append(group_feature)
        actual_f, _ = f_oneway(input_features[0], input_features[1], input_features[2])
        actual_fs.append(float(actual_f))
    output_df['actual_f'] = actual_fs
    output_df['abs_error'] = abs(output_df['actual_f'] - output_df['f'])
    print(filename, output_df['abs_error'].sum() / output_df.shape[0])

c_exact_perm.csv 1.542885159189211e-07
cuda_exact_lexico_perm_anova.csv 1.542885159189211e-07
cuda_exact_rank_perm_anova.csv 1.542885159189211e-07
cuda_shared_exact_rank_perm_anova.csv 1.542885159189211e-07


#### Output Check for Exact Permutation Test

In [18]:
import numpy as np
from scipy.stats import f_oneway

features = pd.read_csv("dataset.csv",header=None)[0].tolist()
filenames = [
    'c_exact_perm.csv', 
    'cuda_exact_lexico_perm_anova.csv', 
    'cuda_exact_rank_perm_anova.csv', 
    'cuda_shared_exact_rank_perm_anova.csv'
]
extreme_counts = []
p_values = []
maes = []

for filename in filenames:
    file_object = open(filename)
    content = file_object.read()
    permutations = []
    f_stats = []
    indices = []
    for i, row in enumerate(content.split('\n')):
        if i < 10:
            permutations.append(row[:-1]) 
        elif i < 20:
            index, f = row.split(',')
            indices.append(int(index))
            f_stats.append(float(f))
        elif row != '':
            extreme_count, p_value = row.split(',')
            extreme_counts.append(int(extreme_count))
            p_values.append(float(p_value))
            
    permutations = np.array(permutations)
    f_stats = np.array(f_stats)
    indices = np.array(indices)
    output_df = pd.DataFrame(np.vstack([indices, permutations, f_stats])).T
    output_df.columns = ['i', 'perm', 'f']
    output_df['i'] = output_df['i'].astype(int)
    output_df['f'] = output_df['f'].astype(float)
    actual_fs = []
    for perm in output_df['perm']:
        permuted_df = pd.DataFrame(np.vstack([np.array(perm.split(',')), features])).T
        permuted_df[0] = permuted_df[0].astype(int)
        permuted_df[1] = permuted_df[1].astype(float)
        keys = permuted_df[0].unique().tolist()
        input_features = []
        for key in keys:
            group_feature = permuted_df[permuted_df[0] == key][1].tolist()
            input_features.append(group_feature)
        actual_f, _ = f_oneway(input_features[0], input_features[1], input_features[2])
        actual_fs.append(float(actual_f))
    output_df['actual_f'] = actual_fs
    output_df['abs_error'] = abs(output_df['actual_f'] - output_df['f'])
    maes.append(output_df['abs_error'].sum() / output_df.shape[0])
filenames = np.array(filenames)
extreme_counts = np.array(extreme_counts)
p_values = np.array(p_values)
maes = np.array(maes)

compiled_df = pd.DataFrame(np.vstack([filenames, extreme_counts, p_values, maes])).T
compiled_df.columns = ['method', 'extremes', 'p-values', 'MAE']
compiled_df['method'] = compiled_df['method'].apply(lambda x: x[:-4])
compiled_df

,method,extremes,p-values,MAE
0,c_exact_perm,5165,0.149062,1.542885159189211e-07
1,cuda_exact_lexico_perm_anova,5165,0.149062,1.542885159189211e-07
2,cuda_exact_rank_perm_anova,5165,0.149062,1.542885159189211e-07
3,cuda_shared_exact_rank_perm_anova,5165,0.149062,1.542885159189211e-07


### Monte Carlo Permutation Test

#### C Version (Serial)

In [19]:
%%writefile c_monte_perm.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define MAX_LINE 1024

/* Linear Congruential Generator (LCG) */
unsigned int lcg_random(unsigned int seed) {
    return (1103515245U * (seed) + 12345U) & 0x7fffffffU;
}

/* Fisher–Yates Shuffling Algorithm */
void permute(size_t *array, size_t N, unsigned int seed, size_t *result) {
    for (size_t i = 0; i < N; i++) {
        result[i] = array[i];
    }
    for (size_t i = N - 1; i > 0; i--) {
        size_t j = lcg_random(seed) % (i + 1);  // pick random index [0, i]
        size_t temp = result[i];
        result[i] = result[j];
        result[j] = temp;
    }
}
/* One Way ANOVA */
double OneWayAnova(size_t N, int k, size_t *n_i, size_t *group, double *feature){
    /* AVERAGE & GROUP AVERAGE */
    double *group_ave = (double *) calloc(k, sizeof(double));
    double average = 0.0;
    for (int i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    for (int i = 0; i < k; i++) {
        group_ave[i] /= n_i[i];
    }

    /* SUM OF SQUARED ERROR (SSE) */
    double SSE = 0.0;
    double temp;
    for (int i = 0; i < N; i++) {
        temp = feature[i] - group_ave[group[i]];
        SSE += temp*temp;
    }

    /* SSR (SUM OF SQUARED RESIDUALS) */
    double SSR = 0.0;
    for (int i = 0; i < k; i++) {
        temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }
    free(group_ave);
    /* F-statistic */
    return (SSR/(k-1))/(SSE/(N-k));
}
int main() {
    size_t perm_count;
    size_t N;   // number of rows
    size_t k;   // number of groups
    clock_t start, end;
    size_t counter = 10;

    /* GET THE NUMBER OF ROWS */
    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);
    printf("Number of Permutations: ");
    scanf("%zu", &perm_count);

    double *feature = (double*) malloc(N * sizeof(double));
    size_t *group = (size_t*) malloc(N * sizeof(size_t));
    size_t *temp_group = (size_t*) malloc(N * sizeof(size_t));
    size_t *perm_array = (size_t*) malloc(N * 10 * sizeof(size_t));
    size_t *n_i = (size_t*) calloc(k, sizeof(size_t));
    double *F_dist = (double*) malloc(perm_count * sizeof(double));

    /* READ THE DATA */
    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL){
        perror("Error opening file");
        return 1;
    }

    char line[MAX_LINE];
    size_t i = 0;
    while (fgets(line, sizeof(line), fp)) {
        if (i >= N) break;  // prevent overflow

        line[strcspn(line, "\n")] = 0;

        char *token = strtok(line, ",");
        int j = 0;
        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token); // convert to float and save
            else {
                group[i] = atoi(token); // convert to int and save
                if (group[i] >= k){
                    perror("Error group count");
                    return 1;
                }
                n_i[group[i]] += 1;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);

    // fill-in cache
    permute(group, N, i, temp_group);
    OneWayAnova(N, k, n_i, group, feature);

    // Execution time start here: CPU Permutation
    /* CPU PERMUTATION */
    double elapse, time_taken;
    elapse = 0.0f;

    for (int c=0; c<counter; c++){
        memcpy(perm_array, group, N * sizeof(size_t));
        size_t p = 1;
        start = clock();
        for (size_t i = 0; i < perm_count; i++) {            
            // Always permute from the ORIGINAL group array
            permute(group, N, i, temp_group);
            if ((i > 0 && i < 5) || (i > perm_count - 6)){
                memcpy(&perm_array[p * N], temp_group, N * sizeof(size_t));
                p++;
            }

            if (i == 0)
                F_dist[i] = OneWayAnova(N, k, n_i, group, feature);
            else 
                F_dist[i] = OneWayAnova(N, k, n_i, temp_group, feature);
        }
        end = clock();
        time_taken = ((double)(end-start))*1E3/CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
    }
    FILE *fptr;
    fptr = fopen("c_monte_perm.csv", "w");

    printf("\nFunction (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", counter, elapse/counter, perm_count);

    // Print first 5 and last 5 permutations
    printf("\nPrinting First 5 and Last 5 Permutations\n");
    for (size_t i = 0; i < 5; i++) {
        printf("CPU Permutation %zu: ", i+1);
        for (size_t j = 0; j < N; j++) {
            printf("%zu ", perm_array[i*N + j]);
            fprintf(fptr, "%zu,", perm_array[i*N + j]);
        }
        printf("\n");
        fprintf(fptr, "\n");
    }
    printf("=================================\n");
    for (size_t i = 5; i < 10; i++) {
        printf("CPU Permutation %zu: ", i+1);
        for (size_t j = 0; j < N; j++) {
            printf("%zu ", perm_array[i*N + j]);
            fprintf(fptr, "%zu,", perm_array[i*N + j]);
        }
        printf("\n");
        fprintf(fptr, "\n");
    }

    printf("\nPrinting First 5 and Last 5 Results\n");
    for (int i = 0; i < 5; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf ("F_dist %d: %lf\n", i+1, F_dist[i]);
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf ("F_dist %d: %lf\n", i+1, F_dist[i]);
    } 

    size_t extreme_count = 0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
       }
    }

    // Calculating the p-value for the permutation test
    double p_value = (double)extreme_count/perm_count;
    printf("\nNull F: %lf\n", F_dist[0]);
    printf ("Extreme Count: %lu\n", extreme_count);
    p_value = (double)extreme_count / perm_count;
    printf("p-value: %lf\n", p_value);

    // saving extreme count and p-value
    fprintf(fptr, "%zu,%lf\n", extreme_count, p_value);
    fclose(fptr);

    // free the allocated memory
    free(feature);
    free(group);
    free(n_i);
    free(F_dist);

    return 0;
}

Overwriting c_monte_perm.c


In [20]:
%%bash
gcc c_monte_perm.c -o c_monte_perm -lm

In [21]:
%%bash
./c_monte_perm < input.txt

Number of Rows: Number of Groups: Number of Permutations: 
Function (in C) average time for 10 loops is 488.007300 milliseconds to execute an array size 1000000 

Printing First 5 and Last 5 Permutations
CPU Permutation 1: 0 0 0 0 1 1 1 1 2 2 2 2 
CPU Permutation 2: 0 0 1 0 1 2 1 2 2 2 0 1 
CPU Permutation 3: 0 2 1 2 2 0 0 0 1 2 1 1 
CPU Permutation 4: 1 2 0 1 1 2 2 0 2 0 0 1 
CPU Permutation 5: 1 0 2 0 1 1 0 2 0 2 2 1 
CPU Permutation 6: 1 1 2 1 2 1 2 2 0 0 0 0 
CPU Permutation 7: 0 0 0 2 1 2 1 1 2 2 1 0 
CPU Permutation 8: 0 0 2 1 1 2 1 0 0 2 1 2 
CPU Permutation 9: 0 1 0 0 2 1 1 1 2 0 2 2 
CPU Permutation 10: 2 0 1 0 0 0 2 1 2 1 1 2 

Printing First 5 and Last 5 Results
F_dist 1: 2.471594
F_dist 2: 1.689050
F_dist 3: 0.368143
F_dist 4: 1.649306
F_dist 5: 2.073082
F_dist 999996: 0.692028
F_dist 999997: 1.486958
F_dist 999998: 0.130352
F_dist 999999: 0.084501
F_dist 1000000: 1.494248

Null F: 2.471594
Extreme Count: 164512
p-value: 0.164512


#### CUDA Version

In [22]:
%%writefile cuda_monte_perm.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define MAX_LINE 1024

/* Linear Congruential Generator (LCG) */
__device__ unsigned int lcg_random(unsigned int seed) {
    return (1103515245U * (seed) + 12345U) & 0x7fffffffU;
}

/* Fisher–Yates Shuffling Algorithm */
__device__ void permute(size_t *array, size_t N, unsigned int seed, size_t *result) {
    for (size_t i = 0; i < N; i++) {
        result[i] = array[i];
    }
    for (size_t i = N - 1; i > 0; i--) {
        size_t j = lcg_random(seed) % (i + 1);
        size_t temp = result[i];
        result[i] = result[j];
        result[j] = temp;
    }
}

/* One Way ANOVA */
__device__ double OneWayAnova(size_t N, int k, size_t *n_i, size_t *group, double *feature){
    
    double group_ave[100];
    for (int i = 0; i < k; i++) {
        group_ave[i] = 0.0;
    }
        
    double average = 0.0;
    for (int i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    for (int i = 0; i < k; i++) {
        group_ave[i] /= n_i[i];
    }

    /* SUM OF SQUARED ERROR (SSE) */
    double SSE = 0.0;
    double temp;
    for (int i = 0; i < N; i++) {
        temp = feature[i] - group_ave[group[i]];
        SSE += temp*temp;
    }

    /* SSR (SUM OF SQUARED RESIDUALS) */
    double SSR = 0.0;
    for (int i = 0; i < k; i++) {
        temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }

    /* F-statistic */
    return (SSR/(k-1))/(SSE/(N-k));
}

__global__ void gpu_permute_and_anova(size_t *array, size_t perm_count, size_t N, int k, 
    double *feature, size_t *n_i, size_t *perm_array, double *F_dist) {
    int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = thread_id; i < perm_count; i += stride) {
        size_t *current_perm = &perm_array[i * N];
        
        if (i == 0) {
            // Copy original data
            for (int j = 0; j < N; j++) {
                current_perm[j] = array[j];
            }
        } else {
            // Generate permutation
            permute(array, N, i, current_perm);
        }
        
        // Compute ANOVA immediately
        F_dist[i] = OneWayAnova(N, k, n_i, current_perm, feature);
    }
}

int main() {
    size_t perm_count;
    size_t N;
    size_t k;
    size_t counter = 10;

    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);
    printf("Number of Permutations: ");
    scanf("%zu", &perm_count);

    // Get GPU device
    int device = -1;
    cudaGetDevice(&device);

    // Memory allocation
    double *feature;
    size_t *group;
    size_t *n_i;
    size_t *perm_array;
    double *F_dist;

    cudaMallocManaged(&feature, N * sizeof(double));
    cudaMallocManaged(&group, N * sizeof(size_t));
    cudaMallocManaged(&n_i, k * sizeof(size_t));
    cudaMallocManaged(&perm_array, N * perm_count * sizeof(size_t));
    cudaMallocManaged(&F_dist, perm_count * sizeof(double));

    // Initialize n_i to zero
    memset(n_i, 0, k * sizeof(size_t));

    // MEMORY ADVISE: Set up for input data (feature, group, n_i)
    cudaMemAdvise(feature, N * sizeof(double), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(group, N * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(n_i, k * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);


    // Prefetch data to CPU memory
    cudaMemPrefetchAsync(feature, N * sizeof(double), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), cudaCpuDeviceId, NULL);

    // READ DATA FROM FILE
    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL){
        perror("Error opening file");
        return 1;
    }

    char line[MAX_LINE];
    size_t i = 0;
    while (fgets(line, sizeof(line), fp)) {
        if (i >= N) break;

        line[strcspn(line, "\n")] = 0;

        char *token = strtok(line, ",");
        int j = 0;
        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token);
            else {
                group[i] = atoi(token);
                if (group[i] >= k){
                    perror("Error group count");
                    fclose(fp);
                    return 1;
                }
                n_i[group[i]] += 1;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);
    
    // PREFETCH: Move input data to GPU before computation
    cudaMemPrefetchAsync(feature, N * sizeof(double), device, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), device, NULL);
    
    // Prefetch output arrays to GPU
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), device, NULL);

    // Wait for prefetch to complete
    cudaDeviceSynchronize();

    // Number of Threads and Blocks
    size_t numThreads = 256;
    size_t numBlocks = (perm_count + numThreads - 1) / numThreads;

    printf("\n Generating Permutations and Computing F-statistic\n");
    printf("Launching kernel with %zu blocks and %zu threads per block\n", numBlocks, numThreads);
    
    for (size_t c = 0; c < counter; c++){
        gpu_permute_and_anova<<<numBlocks, numThreads>>>(
            group, perm_count, N, k, feature, n_i, perm_array, F_dist);
    }
    cudaDeviceSynchronize();

    // PREFETCH: Move results back to CPU for printing
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), cudaCpuDeviceId, NULL);

    FILE *fptr;
    fptr = fopen("cuda_monte_perm.csv", "w");

    printf("\nPrinting First 5 and Last 5 permutations\n");
    for (int i = 0; i < 5; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }

    printf("\nPrinting First 5 and Last 5 F-statistics:\n");
    for (int i = 0; i < 5; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }

    // Calculate p-value
    size_t extreme_count = 0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
        }
    }
    double p_value = (double)extreme_count / (double)perm_count;
    printf("\nNull: %lf\n", F_dist[0]);
    printf("Extreme count: %zu\n", extreme_count);
    printf("p-value: %lf\n", p_value);

    fprintf(fptr, "%zu,%lf", extreme_count, p_value);

    cudaFree(feature);
    cudaFree(group);
    cudaFree(n_i);
    cudaFree(F_dist);

    return 0;
}

Overwriting cuda_monte_perm.cu


In [23]:
%%bash
nvcc cuda_monte_perm.cu -o cuda_monte_perm -Wno-deprecated-gpu-targets

In [24]:
%%bash
nvprof ./cuda_monte_perm < input.txt

==404049== NVPROF is profiling process 404049, command: ./cuda_monte_perm


Number of Rows: Number of Groups: Number of Permutations: 
 Generating Permutations and Computing F-statistic
Launching kernel with 3907 blocks and 256 threads per block

Printing First 5 and Last 5 permutations
GPU Permutation 1: 0 0 0 0 1 1 1 1 2 2 2 2 
GPU Permutation 2: 0 0 1 0 1 2 1 2 2 2 0 1 
GPU Permutation 3: 0 2 1 2 2 0 0 0 1 2 1 1 
GPU Permutation 4: 1 2 0 1 1 2 2 0 2 0 0 1 
GPU Permutation 5: 1 0 2 0 1 1 0 2 0 2 2 1 
GPU Permutation 999996: 1 1 2 1 2 1 2 2 0 0 0 0 
GPU Permutation 999997: 0 0 0 2 1 2 1 1 2 2 1 0 
GPU Permutation 999998: 0 0 2 1 1 2 1 0 0 2 1 2 
GPU Permutation 999999: 0 1 0 0 2 1 1 1 2 0 2 2 
GPU Permutation 1000000: 2 0 1 0 0 0 2 1 2 1 1 2 

Printing First 5 and Last 5 F-statistics:
F_dist 1: 2.471594
F_dist 2: 1.689050
F_dist 3: 0.368143
F_dist 4: 1.649306
F_dist 5: 2.073082
F_dist 999996: 0.692028
F_dist 999997: 1.486958
F_dist 999998: 0.130352
F_dist 999999: 0.084501
F_dist 1000000: 1.494248

Null: 2.471594
Extreme count: 164512
p-value: 0.164512


==404049== Profiling application: ./cuda_monte_perm
==404049== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  36.641ms        10  3.6641ms  1.5127ms  4.6524ms  gpu_permute_and_anova(unsigned long*, unsigned long, unsigned long, int, double*, unsigned long*, unsigned long*, double*)
      API calls:   90.08%  1.26084s         5  252.17ms  75.576us  1.25994s  cudaMallocManaged
                    6.47%  90.551ms        10  9.0551ms  17.260us  80.146ms  cudaMemPrefetchAsync
                    2.76%  38.644ms         2  19.322ms  81.102us  38.563ms  cudaDeviceSynchronize
                    0.27%  3.8340ms         4  958.49us  104.59us  1.8250ms  cudaFree
                    0.23%  3.2124ms        10  321.24us  16.018us  2.9882ms  cudaLaunchKernel
                    0.07%  980.07us       114  8.5970us     152ns  540.19us  cuDeviceGetAttribute
                    0.06%  875.31us         1  875.31us  875.31us  

#### Monte Carlo CUDA Version (Parallel w/ shared)

In [25]:
%%writefile cuda_shared_monte_perm.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define MAX_LINE 1024

/* Linear Congruential Generator (LCG) */
__device__ unsigned int lcg_random(unsigned int seed) {
    return (1103515245U * (seed) + 12345U) & 0x7fffffffU;
}

/* Fisher–Yates Shuffling Algorithm */
__device__ void permute(size_t *array, size_t N, unsigned int seed, size_t *result) {
    for (size_t i = 0; i < N; i++) {
        result[i] = array[i];
    }
    for (size_t i = N - 1; i > 0; i--) {
        size_t j = lcg_random(seed) % (i + 1);
        size_t temp = result[i];
        result[i] = result[j];
        result[j] = temp;
    }
}

/* One Way ANOVA */
__device__ double OneWayAnova(size_t N, int k, size_t *n_i, size_t *group, double *feature){
    
    double group_ave[100];
    for (int i = 0; i < k; i++) {
        group_ave[i] = 0.0;
    }
        
    double average = 0.0;
    for (int i = 0; i < N; i++) {
        group_ave[group[i]] += feature[i];
        average += feature[i];
    }
    average /= N;
    for (int i = 0; i < k; i++) {
        group_ave[i] /= n_i[i];
    }

    /* SUM OF SQUARED ERROR (SSE) */
    double SSE = 0.0;
    double temp;
    for (int i = 0; i < N; i++) {
        temp = feature[i] - group_ave[group[i]];
        SSE += temp*temp;
    }

    /* SSR (SUM OF SQUARED RESIDUALS) */
    double SSR = 0.0;
    for (int i = 0; i < k; i++) {
        temp = group_ave[i] - average;
        SSR += n_i[i] * (temp * temp);
    }

    /* F-statistic */
    return (SSR/(k-1))/(SSE/(N-k));
}


__device__ void gpu_anova(size_t *perm_array, size_t N, int k, size_t perm_count, double *feature, size_t *n_i, double *F_dist) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int perm_idx = idx; perm_idx < perm_count; perm_idx += stride) {
        size_t *current_group = &perm_array[perm_idx * N];
        F_dist[perm_idx] = OneWayAnova(N, k, n_i, current_group, feature);
    }
}

__global__ void gpu_permute_and_anova(size_t *array, size_t perm_count, size_t N, int k, 
    double *feature, size_t *n_i, size_t *perm_array, double *F_dist) {
    extern __shared__ char shared_mem[];
    size_t* shared_group = (size_t*)shared_mem;
    double* shared_feature = (double*)(shared_mem + N * sizeof(size_t));

    int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    int lindex = threadIdx.x;

    // Load data into shared memory
    for (int i = lindex; i < N; i += blockDim.x) {
        shared_group[i] = array[i];
        shared_feature[i] = feature[i];
    }

    __syncthreads();

    int stride = blockDim.x * gridDim.x;
    for (int i = thread_id; i < perm_count; i += stride) {
        size_t *current_perm = &perm_array[i * N];
        
        if (i == 0) {
            // Copy original data
            for (int j = 0; j < N; j++) {
                current_perm[j] = shared_group[j];
            }
        } else {
            // Generate permutation
            permute(shared_group, N, i, current_perm);
        }
        F_dist[i] = OneWayAnova(N, k, n_i, current_perm, feature);
    }
}

int main() {
    size_t perm_count;
    size_t N;
    size_t k;
    size_t counter = 10;

    printf("Number of Rows: ");
    scanf("%zu", &N);
    printf("Number of Groups: ");
    scanf("%zu", &k);
    printf("Number of Permutations: ");
    scanf("%zu", &perm_count);

    // Get GPU device
    int device = -1;
    cudaGetDevice(&device);

    // Memory allocation
    double *feature;
    size_t *group;
    size_t *n_i;
    size_t *perm_array;
    double *F_dist;

    cudaMallocManaged(&feature, N * sizeof(double));
    cudaMallocManaged(&group, N * sizeof(size_t));
    cudaMallocManaged(&n_i, k * sizeof(size_t));
    cudaMallocManaged(&perm_array, N * perm_count * sizeof(size_t));
    cudaMallocManaged(&F_dist, perm_count * sizeof(double));

    // Initialize n_i to zero
    memset(n_i, 0, k * sizeof(size_t));

    // MEMORY ADVISE: Set up for input data (feature, group, n_i)
    cudaMemAdvise(feature, N * sizeof(double), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(group, N * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(n_i, k * sizeof(size_t), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);

    // Prefetch data to CPU memory
    cudaMemPrefetchAsync(feature, N * sizeof(double), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), cudaCpuDeviceId, NULL);


    // READ DATA FROM FILE
    FILE *fp = fopen("dataset.csv", "r");
    if (fp == NULL){
        perror("Error opening file");
        return 1;
    }

    char line[MAX_LINE];
    size_t i = 0;
    while (fgets(line, sizeof(line), fp)) {
        if (i >= N) break;

        line[strcspn(line, "\n")] = 0;

        char *token = strtok(line, ",");
        int j = 0;
        while (token != NULL) {
            if (j == 0)
                feature[i] = atof(token);
            else {
                group[i] = atoi(token);
                if (group[i] >= k){
                    perror("Error group count");
                    fclose(fp);
                    return 1;
                }
                n_i[group[i]] += 1;
            }
            token = strtok(NULL, ",");
            j++;
        }
        i++;
    }
    fclose(fp);
    
    // PREFETCH: Move input data to GPU before computation
    cudaMemPrefetchAsync(feature, N * sizeof(double), device, NULL);
    cudaMemPrefetchAsync(group, N * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(n_i, k * sizeof(size_t), device, NULL);
    
    // Prefetch output arrays to GPU
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), device, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), device, NULL);

    // Wait for prefetch to complete
    cudaDeviceSynchronize();

    // Number of Threads and Blocks
    size_t numThreads = 256;
    size_t numBlocks = (perm_count + numThreads - 1) / numThreads;

    printf("\n Generating Permutations and Computing F-statistic\n");
    printf("Launching kernel with %zu blocks and %zu threads per block\n", numBlocks, numThreads);
    
    for (size_t c = 0; c < counter; c++){
        gpu_permute_and_anova<<<numBlocks, numThreads, N*2>>>(
            group, perm_count, N, k, feature, n_i, perm_array, F_dist);
    }
    cudaDeviceSynchronize();

    // PREFETCH: Move results back to CPU for printing
    cudaMemPrefetchAsync(perm_array, N * perm_count * sizeof(size_t), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(F_dist, perm_count * sizeof(double), cudaCpuDeviceId, NULL);

    FILE *fptr;
    fptr = fopen("cuda_shared_monte_perm.csv", "w");

    printf("\nPrinting First 5 and Last 5 permutations\n");
    for (int i = 0; i < 5; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        printf("GPU Permutation %d: ", i + 1);
        for (int j = 0; j < N; j++) {
            fprintf(fptr, "%zu,", perm_array[i * N + j]);
            printf("%zu ", perm_array[i * N + j]);
        }
        fprintf(fptr, "\n");
        printf("\n");
    }

    printf("\nPrinting First 5 and Last 5 F-statistics:\n");
    for (int i = 0; i < 5; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }
    printf("=================================\n");
    for (int i = perm_count-5; i < perm_count; i++) {
        fprintf(fptr, "%d,%lf\n", i, F_dist[i]);
        printf("F_dist %d: %lf\n", i+1, F_dist[i]);
    }

    // Calculate p-value
    size_t extreme_count = 0;
    for (size_t i = 1; i < perm_count; i++) {
        if (F_dist[i] >= F_dist[0]) {
            extreme_count++;
        }
    }
    double p_value = (double)extreme_count / (double)perm_count;
    printf("\nNull: %lf\n", F_dist[0]);
    printf("Extreme count: %zu\n", extreme_count);
    printf("p-value: %lf\n", p_value);

    fprintf(fptr, "%zu,%lf", extreme_count, p_value);
    
    // Free memory
    cudaFree(feature);
    cudaFree(group);
    cudaFree(n_i);
    cudaFree(perm_array);
    cudaFree(F_dist);

    return 0;
}

Overwriting cuda_shared_monte_perm.cu


In [26]:
%%bash
nvcc cuda_shared_monte_perm.cu -o cuda_shared_monte_perm -Wno-deprecated-gpu-targets

In [27]:
%%bash
nvprof ./cuda_shared_monte_perm < input.txt

==404108== NVPROF is profiling process 404108, command: ./cuda_shared_monte_perm


Number of Rows: Number of Groups: Number of Permutations: 
 Generating Permutations and Computing F-statistic
Launching kernel with 3907 blocks and 256 threads per block

Printing First 5 and Last 5 permutations
GPU Permutation 1: 0 0 0 0 1 1 1 1 2 2 2 2 
GPU Permutation 2: 0 0 1 0 1 2 1 2 2 2 0 1 
GPU Permutation 3: 0 2 1 2 2 0 0 0 1 2 1 1 
GPU Permutation 4: 1 2 0 1 1 2 2 0 2 0 0 1 
GPU Permutation 5: 1 0 2 0 1 1 0 2 0 2 2 1 
GPU Permutation 999996: 1 1 2 1 2 1 2 2 0 0 0 0 
GPU Permutation 999997: 0 0 0 2 1 2 1 1 2 2 1 0 
GPU Permutation 999998: 0 0 2 1 1 2 1 0 0 2 1 2 
GPU Permutation 999999: 0 1 0 0 2 1 1 1 2 0 2 2 
GPU Permutation 1000000: 2 0 1 0 0 0 2 1 2 1 1 2 

Printing First 5 and Last 5 F-statistics:
F_dist 1: 2.471594
F_dist 2: 1.689050
F_dist 3: 0.368143
F_dist 4: 1.649306
F_dist 5: 2.073082
F_dist 999996: 0.692028
F_dist 999997: 1.486958
F_dist 999998: 0.130352
F_dist 999999: 0.084501
F_dist 1000000: 1.494248

Null: 2.471594
Extreme count: 164512
p-value: 0.164512


==404108== Profiling application: ./cuda_shared_monte_perm
==404108== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  36.424ms        10  3.6424ms  1.5147ms  4.6016ms  gpu_permute_and_anova(unsigned long*, unsigned long, unsigned long, int, double*, unsigned long*, unsigned long*, double*)
      API calls:   83.93%  861.01ms         5  172.20ms  49.850us  860.39ms  cudaMallocManaged
                   11.04%  113.28ms        10  11.328ms  19.046us  101.25ms  cudaMemPrefetchAsync
                    3.77%  38.641ms         2  19.320ms  46.069us  38.594ms  cudaDeviceSynchronize
                    0.84%  8.5792ms         5  1.7158ms  84.357us  5.8009ms  cudaFree
                    0.31%  3.1513ms        10  315.13us  7.1370us  3.0501ms  cudaLaunchKernel
                    0.04%  376.34us       114  3.3010us     104ns  167.75us  cuDeviceGetAttribute
                    0.04%  368.04us         3  122.68us  9.5

#### Output Check for Exact Permutation Test

In [28]:
import numpy as np
from scipy.stats import f_oneway

features = pd.read_csv("dataset.csv",header=None)[0].tolist()
filenames = [
    'c_monte_perm.csv', 
    'cuda_monte_perm.csv', 
    'cuda_shared_monte_perm.csv'
]
extreme_counts = []
p_values = []
maes = []

for filename in filenames:
    file_object = open(filename)
    content = file_object.read()
    permutations = []
    f_stats = []
    indices = []
    for i, row in enumerate(content.split('\n')):
        if i < 10:
            permutations.append(row[:-1]) 
        elif i < 20:
            index, f = row.split(',')
            indices.append(int(index))
            f_stats.append(float(f))
        elif row != '':
            extreme_count, p_value = row.split(',')
            extreme_counts.append(int(extreme_count))
            p_values.append(float(p_value))
            
    permutations = np.array(permutations)
    f_stats = np.array(f_stats)
    indices = np.array(indices)
    output_df = pd.DataFrame(np.vstack([indices, permutations, f_stats])).T
    output_df.columns = ['i', 'perm', 'f']
    output_df['i'] = output_df['i'].astype(int)
    output_df['f'] = output_df['f'].astype(float)
    actual_fs = []
    for perm in output_df['perm']:
        permuted_df = pd.DataFrame(np.vstack([np.array(perm.split(',')), features])).T
        permuted_df[0] = permuted_df[0].astype(int)
        permuted_df[1] = permuted_df[1].astype(float)
        keys = permuted_df[0].unique().tolist()
        input_features = []
        for key in keys:
            group_feature = permuted_df[permuted_df[0] == key][1].tolist()
            input_features.append(group_feature)
        actual_f, _ = f_oneway(input_features[0], input_features[1], input_features[2])
        actual_fs.append(float(actual_f))
    output_df['actual_f'] = actual_fs
    output_df['abs_error'] = abs(output_df['actual_f'] - output_df['f'])
    maes.append(output_df['abs_error'].sum() / output_df.shape[0])
filenames = np.array(filenames)
extreme_counts = np.array(extreme_counts)
p_values = np.array(p_values)
maes = np.array(maes)

compiled_df = pd.DataFrame(np.vstack([filenames, extreme_counts, p_values, maes])).T
compiled_df.columns = ['method', 'extremes', 'p-values', 'MAE']
compiled_df['method'] = compiled_df['method'].apply(lambda x: x[:-4])
compiled_df

,method,extremes,p-values,MAE
0,c_monte_perm,164512,0.164512,2.74256317564503e-07
1,cuda_monte_perm,164512,0.164512,2.74256317564503e-07
2,cuda_shared_monte_perm,164512,0.164512,2.74256317564503e-07
